# 06 Compute the volume of each tissue

The volume is just the number of voxels with density larger than 10 corresponding to each tissue
- Whole fruit
- Exocarp
- Mesocarp
- Rind
- Endocarp
- Central column or spine
- Oil gland tissue

We will convert voxel size into standard volume units later with additional scanning metadata.

Additionally, there is the unitless number of individual oil glands

For each fruit, we create a 10-dim vector
```
WR05_CRC3605_18B-19-5 , L03 , 30638950 , 8584712 , 18614789 , 5900155 , 373572 , 5502226 , 290577 , 5018
```
corresponding to 
```
1.  root name
2.  label name
3.  whole size
4.  exocarp size
5.  endocarp size
6.  rind size
7.  spine size
8.  mesocarp size
9.  oil gland tissue size
10. number of individual oil glands
```

All the results will be stored in `data/allometry/data` path following our established naming conventions.
- The idea is to generate an individual file per fruit in a single folder
- All data can easily be compiled later by doing on the terminal
```
cat *.csv > ../citrus_sizes.csv
```

The file `data/allometry/citrus_sizes.csv` is available with the voxel-size information of 161 different fruit samples comprising 50 citrus varieties.

In [1]:
import os
import sys
import glob

import scipy.ndimage as ndimage
import scipy.spatial as spatial
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings( "ignore")
import matplotlib.pyplot as plt
%matplotlib inline
from importlib import reload

import tifffile as tf
import citrus_utils as vitaminC

### Load the data of one fruit

In [2]:
comps_src = '../data/comps/'
tissue_src = '../data/tissue/'
oil_src = '../data/oil/'
bname = 'WR05_CRC3605_18B-19-5'

In [8]:
src = comps_src + bname
comp_files = sorted(glob.glob(src + '*.tif'))
print(comp_files)

i = 0 
fname = os.path.split(comp_files[i])[1]
lname = os.path.splitext(fname)[0].split('_')[-1]
print('\n',lname)
#for lname in os.listdir(src):

tissue_filename = tissue_src + bname + '/' + lname + '/' + bname + '_' + lname + '_'
print(tissue_filename)

['../data/comps/WR05_CRC3605_18B-19-5_L03.tif']

 L03
../data/tissue/WR05_CRC3605_18B-19-5/L03/WR05_CRC3605_18B-19-5_L03_


In [9]:
tissue_size = []
tissue_size.append(bname)
tissue_size.append(lname)
tissue = tf.imread(comp_files[i])
tissue_size.append(np.sum(tissue > 10))

In [10]:
tissue_types = ['exocarp', 'endocarp', 'rind']

for tss in tissue_types:
    tissue = tf.imread(tissue_filename + tss + '.tif')
    tissue_size.append(np.sum(tissue > 10))

In [11]:
tissue_size

['WR05_CRC3605_18B-19-5', 'L03', 30638950, 8584712, 18614789, 5900155]

In [12]:
spine = tf.imread(tissue_filename + 'spine.tif')
tissue_size.append(np.sum(spine > 10))

In [13]:
spine[spine > 0] = 1
meso = tissue*(1 - spine)
tendo = vitaminC.get_largest_element(meso)
meso *= tendo

393 components
[5502226    4214    2209    1993    1748    1388    1097    1001     945
     819     678     668     519     429     307     289     229     226
     225     221]


In [14]:
tissue_size.append(np.sum(meso > 10))

In [15]:
citrus_oil = oil_src + bname + '/' + lname + '/' + bname + '_'
tissue = tf.imread(citrus_oil + 'glands.tif')
tissue_size.append(np.sum(tissue > 10))

The number of individual oil glands is just the number of lines in the `(x,y,z)`-coordinate file

In [16]:
tissue_size.append(vitaminC.buf_count_newlines_gen(citrus_oil + 'glandscenter.csv'))

Save the 10-dimensional vector as a csv

In [18]:
dst = '../data/allometry/data/'
if not os.path.isdir(dst):
    os.makedirs(dst)

df = pd.DataFrame(tissue_size).T
filename = dst + bname + '_' + lname + '.csv'
df.to_csv(filename, index = False, header=False)

In [19]:
df

,0,1,2,3,4,5,6,7,8,9
0,WR05_CRC3605_18B-19-5,L03,30638950,8584712,18614789,5900155,373572,5502226,290577,5018
